In [ ]:
# import basic packages 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

#import warnings
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



# Load Datasets

In [ ]:
train=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
train.head()

In [ ]:
# shape of train and test
print('Shape of train',train.shape)
print('Shape of test',test.shape)

In [ ]:
# split data into target and features
y = train["label"]
X = train.drop(labels=["label"],axis=1)

In [ ]:
# count of values in target feature
plt.figure(figsize=(12,6))
sns.countplot(y)



In [ ]:
y.value_counts()

In [ ]:
# lets check one picture
plt.figure(figsize=(8,6))
img=X.iloc[200].to_numpy()
img=img.reshape((28,28))
plt.imshow(img,cmap='gray')
plt.title(train.iloc[200,0])
plt.grid()
plt.show()

# Normalization

In [ ]:
# Normalize the data
X = X / 255.0
test = test / 255.0
print("X shape: ",X.shape)
print("test shape: ",test.shape)

# Reshape

In [ ]:
# Reshape 
X = X.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
print("X shape: ",X.shape)
print("test shape: ",test.shape)

# Label Encoding

In [ ]:
from keras.utils.np_utils import to_categorical
y = to_categorical(y, num_classes = 10)
y

# Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)
print("x_train shape",X_train.shape)
print("x_test shape",X_test.shape)
print("y_train shape",y_train.shape)
print("y_test shape",y_test.shape)

# Model

In [ ]:
# model making using Keras
from tensorflow.keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

model= Sequential()
model.add(Conv2D(filters=8, kernel_size=(5,5), padding='Same', activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=16, kernel_size=(3,3), padding='Same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.25))

# Fully Connected
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

##### **Optimizers**: Optimizers are algorithms or methods used to change the attributes of your neural network such as weights and learning rate in order to reduce the losses.

In [ ]:
# Define the optimizer
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
epochs = 100  
batch_size = 250

### Data Augmentation

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False, 
        rotation_range=5,  
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=False,  
        vertical_flip=False) 

datagen.fit(X_train)

In [ ]:
# Early stopping

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=6)

In [ ]:
# fit the model

history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_test,y_test), steps_per_epoch=X_train.shape[0] // batch_size,
                              callbacks=[early_stopping])

# Model Evaluation

In [ ]:
# model Success
print("Accuracy of the model is --> " , model.evaluate(X_test, y_test, batch_size=batch_size)[1]*100 , "%")
print("Loss of the model is --> " , model.evaluate(X_test, y_test, batch_size=batch_size)[0])

In [ ]:
# Loss Chart
plt.figure()
plt.plot(history.history["loss"],label="Train Loss",color='black')
plt.plot(history.history["val_loss"],label = "Validation Loss", color = "darkred", marker = "+", linestyle="dashed",markeredgecolor = "purple", markeredgewidth = 2)
plt.title("Model Loss", color = "darkred", size = 13)
plt.legend()
plt.show()

In [ ]:
# Accuracy chart
plt.figure()
plt.plot(history.history["accuracy"],label = "Train Accuracy", color = "black")
plt.plot(history.history["val_accuracy"],label = "Validation Accuracy", color = "darkred", marker = "+", linestyle="dashed",markeredgecolor = "purple", markeredgewidth = 2)
plt.title("Model Accuracy", color = "darkred", size = 13)
plt.legend()
plt.show()

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix

Y_pred = model.predict(X_test)
# argmax = To briefly mention it, it will give the index of the value with the highest value.
Y_pred_classes = np.argmax(Y_pred,axis = 1) 

# We do the same for the y_val values. because we will compare these values. 
Y_true = np.argmax(y_test,axis = 1)
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 

# define the colors


# plot the confusion matrix
f,ax = plt.subplots(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap='Greens',linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label", color = "blue")
plt.ylabel("True Label", color = "red")
plt.title("Confusion Matrix", color = "darkred", size = 15)
plt.show()